In [3]:
import os, sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
os.chdir(parent_dir)

import json
from PIL import Image

import numpy as np
import torch 
from torch.utils.data import Dataset
from tqdm import tqdm
import sys
import pdb
from conf.config import GlobalConfig

/home/zhk/anaconda3/envs/csg/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
ids = {'None':0, 'Buildings':1, 'Fences':2, 'Other':3, 'Pedestrians':4, 'Pole':5, 'RoadLines':6, 'Roads':7, 'Sidewalks':8, 'Vegetation':9, 'Vehicles':10, 'Walls':11, 'TrafficSigns':12, 'Sky':13, 'Ground': 14, 'Bridge': 15, 'RailTrack': 16, 'GuardRail':17, 'TrafficLight':18, 'Static':19, 'Dynamic':20, 'Water':21, 'Terrain':22}
id_obstacle = [4,10]
id_road = [3,6,7,27]
id_traffic_red_light = [23]
config = GlobalConfig()

In [5]:
def img_pre_processing(image, scale=1, crop=256, shift_x=0, shift_y=0):
    return torch.from_numpy(np.array(scale_and_crop_image(Image.open(image), scale=scale, crop=crop, shift_x=shift_x, shift_y=shift_y)))


In [6]:
def seg_pre_processing(seg, scale=1, crop=256, shift_x=0, shift_y=0):
    t = np.array(scale_and_crop_seg(Image.open(seg), scale=scale, crop=crop, shift_x=shift_x, shift_y=shift_y))
    return torch.from_numpy(t)

In [7]:
def scale_and_crop_seg(seg, scale=1, crop=256, shift_x=0, shift_y=0):
    """
    Scale and crop a PIL image, returning a channels-first numpy array.
    """
    # image = Image.open(filename)
    (width, height) = (int(seg.width // scale), int(seg.height // scale))
    seg_resized = seg.resize((width, height))
    seg = np.asarray(seg_resized)
    start_x = height//2 - crop//2 + shift_x
    start_y = width//2 - crop//2 + shift_y
    cropped_seg = seg[start_x:start_x+crop, start_y:start_y+crop]
    cropped_seg = cropped_seg
    return cropped_seg

In [8]:
def scale_and_crop_image(image, scale=1, crop=256, shift_x=0, shift_y=0):
    """
    Scale and crop a PIL image, returning a channels-first numpy array.
    """
    # image = Image.open(filename)
    (width, height) = (int(image.width // scale), int(image.height // scale))
    im_resized = image.resize((width, height))
    image = np.asarray(im_resized)
    start_x = height//2 - crop//2 + shift_x
    start_y = width//2 - crop//2 + shift_y
    cropped_image = image[start_x:start_x+crop, start_y:start_y+crop]
    cropped_image = np.transpose(cropped_image, (2,0,1))
    return cropped_image

In [9]:
def stich_rgb_images(route_dir, filename):
    rgb_front_path = os.path.join(route_dir,"rgb_front",filename)
    rgb_left_path = os.path.join(route_dir,"rgb_left",filename)
    rgb_right_path = os.path.join(route_dir,"rgb_right",filename)

    rgb_front = img_pre_processing(rgb_front_path, scale * 1.1, input_resolution)
    rgb_left = img_pre_processing(rgb_left_path, scale, input_resolution, shift_y=-55)
    rgb_right = img_pre_processing(rgb_right_path, scale * 1.1, input_resolution, shift_y=55)
    
    rgb_lfr = torch.cat((rgb_left, rgb_front, rgb_right), axis=2)
    im = Image.fromarray(rgb_lfr.permute(1, 2, 0).numpy())
    lfr_path = os.path.join(route_dir,"rgb_lfr")
    if not os.path.exists(lfr_path):
        os.makedirs(lfr_path)
    im.save(os.path.join(lfr_path,filename))

In [10]:
def stich_seg_images(route_dir, filename):
    seg_front_path = os.path.join(route_dir,"seg_front",filename)
    seg_left_path = os.path.join(route_dir,"seg_left",filename)
    seg_right_path = os.path.join(route_dir,"seg_right",filename)

    seg_front = seg_pre_processing(seg_front_path, scale * 1.1, input_resolution)
    seg_left = seg_pre_processing(seg_left_path, scale, input_resolution, shift_y=-55)
    seg_right = seg_pre_processing(seg_right_path, scale * 1.1, input_resolution, shift_y=55)
    seg_lfr = torch.cat((seg_front, seg_left, seg_right), axis=1)
    im = Image.fromarray(seg_lfr.numpy())
    lfr_path = os.path.join(route_dir,"seg_lfr")
    if not os.path.exists(lfr_path):
        os.makedirs(lfr_path)
    im.save(os.path.join(lfr_path,filename))

In [11]:
def transform_2d_points(xyz, r1, t1_x, t1_y, r2, t2_x, t2_y):
    """
    Build a rotation matrix and take the dot product.
    """
    # z value to 1 for rotation
    xy1 = xyz.copy()
    xy1[:,2] = 1

    c, s = np.cos(r1), np.sin(r1)
    r1_to_world = np.matrix([[c, s, t1_x], [-s, c, t1_y], [0, 0, 1]])

    # np.dot converts to a matrix, so we explicitly change it back to an array
    world = np.asarray(r1_to_world @ xy1.T)

    c, s = np.cos(r2), np.sin(r2)
    r2_to_world = np.matrix([[c, s, t2_x], [-s, c, t2_y], [0, 0, 1]])
    world_to_r2 = np.linalg.inv(r2_to_world)

    out = np.asarray(world_to_r2 @ world).T
    
    # reset z-coordinate
    out[:,2] = xyz[:,2]

    return out

In [12]:
def lidar_to_histogram_features(lidar, crop=256):
    """
    Convert LiDAR point cloud into 2-bin histogram over 256x256 grid
    """
    def splat_points(point_cloud):
        # 256 x 256 grid
        pixels_per_meter = 8
        hist_max_per_pixel = 5
        x_meters_max = 16
        y_meters_max = 32
        xbins = np.linspace(-2*x_meters_max, 2*x_meters_max+1, 2*x_meters_max*pixels_per_meter+1)
        ybins = np.linspace(-y_meters_max, 0, y_meters_max*pixels_per_meter+1)
        hist = np.histogramdd(point_cloud[...,:2], bins=(xbins, ybins))[0]
        hist[hist>hist_max_per_pixel] = hist_max_per_pixel
        overhead_splat = hist/hist_max_per_pixel
        return overhead_splat

    below = lidar[lidar[...,2]<=-2.0]
    above = lidar[lidar[...,2]>-2.0]
    below_features = splat_points(below)
    above_features = splat_points(above)
    features = np.stack([below_features, above_features], axis=-1)
    features = np.flip(np.transpose(features, (2, 1, 0)), 2).astype(np.float32)
    return features


In [19]:
def lidar_preprocessing(route_dir, filename):
    lidar_path = os.path.join(route_dir,"lidar",filename)
    lidar_unprocessed = np.load(lidar_path)[...,:3] # lidar: XYZI
    lidar_unprocessed[:,1] *= -1
    lidar_processed = lidar_to_histogram_features(lidar_unprocessed, crop=256)
    path = os.path.join(route_dir,"lidar_p")
    if not os.path.exists(path):
        os.makedirs(path)
    np.save(os.path.join(path,filename), lidar_processed)

    

In [20]:
input_resolution = 256
scale = 1.0
def stich_images(config):
    for sub_root in tqdm(config.train_data, file=sys.stdout):
        root_files = os.listdir(sub_root)
        routes = [folder for folder in root_files if not os.path.isfile(os.path.join(sub_root,folder))]
        for route in routes:
            route_dir = os.path.join(sub_root, route)
            num = len(os.listdir(route_dir+"/rgb_front/"))
            for i in range(num):
                filename = f"{str(i).zfill(4)}.png"
                lidar_filename = f"{str(i).zfill(4)}.npy"
                
                stich_rgb_images(route_dir, filename)
                stich_seg_images(route_dir, filename)
                lidar_preprocessing(route_dir, lidar_filename)




In [21]:
stich_images(config=config)

  0%|          | 0/14 [00:00<?, ?it/s]torch.Size([256, 256])
torch.Size([256, 767])
torch.Size([256, 256])
torch.Size([256, 767])
torch.Size([256, 256])
torch.Size([256, 767])
torch.Size([256, 256])
torch.Size([256, 767])
torch.Size([256, 256])
torch.Size([256, 767])
torch.Size([256, 256])
torch.Size([256, 767])
torch.Size([256, 256])
torch.Size([256, 767])
torch.Size([256, 256])
torch.Size([256, 767])
torch.Size([256, 256])
torch.Size([256, 767])
  0%|          | 0/14 [00:06<?, ?it/s]


KeyboardInterrupt: 